In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-08"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
15853,2024-03-08,Israel Superliga,09:30,Hapoel Eilat,Hapoel Tel-Aviv,3.97,1.23,174.5,1.80,1.86,0.0,0.00,0.00,6NpOv2cL,0.251889,0.813008,0.555556,0.537634,0.064897,385.706,257.365989,0.667259,1.8,1.643168,0.912871,390.60,4.924,3.402401,0.690983,-14.0,130.194,39.871323,0.306245,1.8,1.643168,0.912871,117.81,1.484,0.612683,0.412859,18.0,70,99,5.58,1.19,202.218,145.764,0.745182,0.023184,NaN,-3.03,-0.606,-4.900990,0.000000,0.0,0.000000,-0.80,-0.160,-1.437500,0.000000,0.0,0.000000
15854,2024-03-08,Israel Liga Leumit,07:30,Ironi Nahariya,Elitzur Shomron,1.48,2.53,172.5,1.85,1.85,0.0,0.00,0.00,OO8QIg1B,0.675676,0.395257,0.540541,0.540541,0.070933,125.886,22.299125,0.177137,1.2,1.643168,1.369306,93.15,1.578,0.186199,0.117997,-9.0,193.000,67.679949,0.350673,1.8,1.643168,0.912871,220.11,2.144,0.785162,0.366214,10.0,69,87,1.35,2.53,0.000,0.000,0.370305,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
15855,2024-03-08,Israel Liga Leumit,07:45,Elitzor Netania,Elitzur Ashkelon,1.56,2.32,159.5,1.85,1.85,0.0,0.00,0.00,ObwVFioU,0.641026,0.431034,0.540541,0.540541,0.072060,128.904,53.999350,0.418911,2.4,1.341641,0.559017,127.28,1.670,0.447605,0.268027,10.0,135.474,25.549315,0.188592,1.8,1.643168,0.912871,137.64,1.576,0.403894,0.256278,58.0,86,93,1.48,1.48,126.266,0.000,0.277011,0.000000,NaN,0.05,0.010,56.000000,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
15856,2024-03-08,Itália Série A2,16:45,Cento,Agrigento,1.32,3.20,148.5,1.87,1.79,0.0,0.00,0.00,Mil6FS4E,0.757576,0.312500,0.534759,0.558659,0.070076,155.064,42.410948,0.273506,1.8,1.643168,0.912871,165.60,2.028,0.632866,0.312064,-15.0,187.128,43.681066,0.233429,0.0,0.000000,NaN,211.40,2.864,0.641272,0.223908,-60.0,60,70,2.76,3.02,112.858,217.250,0.588213,0.030912,NaN,-0.23,-0.046,-6.956522,0.000000,0.0,0.000000,-5.00,-1.000,-2.200000,0.363577,0.0,-0.363577
15857,2024-03-08,Japão B2.League,07:00,Ehime Orange Vikings,Niigata,1.30,3.34,157.5,1.87,1.79,0.0,0.00,0.00,vasVFNfd,0.769231,0.299401,0.534759,0.558659,0.068632,297.934,142.659982,0.478831,0.6,1.341641,2.236068,470.64,4.308,2.370711,0.550304,-52.0,475.764,283.941968,0.596813,0.0,0.000000,NaN,738.75,6.430,3.981953,0.619277,-127.0,74,75,6.36,9.85,251.258,446.912,0.621766,0.030912,NaN,-1.25,-0.250,-1.200000,0.402483,0.3,-0.102483,-2.16,-0.432,-5.416667,0.206945,0.1,-0.106945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15985,2024-03-08,Eua Ncaa,00:00,UC Riverside,CS Fullerton,1.67,2.28,137.5,1.91,1.91,-4.5,2.05,3.20,jVMRCOhS,0.598802,0.438596,0.523560,0.523560,0.037399,148.122,33.657502,0.227228,1.8,1.643168,0.912871,131.40,2.050,0.639414,0.311909,5.0,195.018,121.730014,0.624199,1.8,1.643168,0.912871,154.05,2.748,1.706962,0.621165,20.0,73,79,1.80,1.95,124.002,204.866,0.218398,0.000000,0.309780,-1.87,-0.374,-1.791444,0.591320,0.5,-0.091320,-0.12,-0.024,-53.333333,0.329318,0.4,0.070682
15986,2024-03-08,Eua Ncaa,00:00,UC San Diego,UC Davis,1.37,3.29,140.5,1.91,1.91,0.0,0.00,0.00,f3TIEpOF,0.729927,0.303951,0.523560,0.523560,0.033878,117.862,36.799327,0.312224,2.4,1.341641,0.559017,122.45,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
15860,16:00,Libano Divisão 1,Hoops,Champville,2.68,Back Home
15898,18:30,Eua Ncaa,Elon,Hampton,1.61,Back Home
15911,06:00,Taiwan Sbl,Yulon Luxgen Dinos,Taiwan Beer,2.68,Back Home
15945,08:35,China Cba,Tianjin,Shenzhen,2.74,Back Home
15947,21:30,Eua Nba,New York Knicks,Orlando Magic,1.74,Back Home
15960,22:00,Eua Ncaa,Oral Roberts,South Dakota Coyotes,1.64,Back Home
15970,16:00,Eua Ncaa,Winthrop,Longwood,1.90,Back Home
